# 2018-2019 Player Classification

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def scrape_basketballreference(url):
    html = urlopen(url)
    soup = BeautifulSoup(html)
    soup.findAll('tr', limit=2)
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    headers = headers[1:]
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    return stats

In [3]:
totals = scrape_basketballreference('https://www.basketball-reference.com/leagues/NBA_2019_totals.html')
per_game = scrape_basketballreference('https://www.basketball-reference.com/leagues/NBA_2019_per_game.html')
advanced = scrape_basketballreference('https://www.basketball-reference.com/leagues/NBA_2019_advanced.html')

In [4]:
#append "_pg" to the appropriate column headers in per_game